In [18]:
import sys
import os
import pandas as pd
import numpy as np

# Append the absolute path of the root directory of the project
notebook_path = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_path, '..', '..'))

# Add the root directory to sys.path
sys.path.append(project_root)

from src.data.preprocess import DataPreprocessor
from src.models.train_model import ModelTrainer
from src.models.predict_model import ModelPredictor
from sklearn.metrics import classification_report, accuracy_score
from src.config import *

In [12]:
file_path = TEST_DIR

# Read the CSV
df = pd.read_csv(file_path)
preprocessor = DataPreprocessor(file_path)
preprocessor.clean_data()
prep_path = os.path.join(PROCESSED_DATA_PATH, 'preprocess.csv')
preprocessor.df.to_csv(prep_path, index = False)
preprocessor.df

,Sentiment,News Headline,clean_text,sentiment_encoded
0,neutral,"According to Gran , the company has no plans t...",according gran company plan move production ru...,1
1,neutral,Technopolis plans to develop in stages an area...,technopolis plan develop stage area less squar...,1
2,negative,The international electronic industry company ...,international electronic industry company elco...,0
3,positive,With the new production plant the company woul...,new production plant company would increase ca...,2
4,positive,According to the company 's updated strategy f...,according company updated strategy year baswar...,2
...,...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,london marketwatch share price ended lower lon...,0
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,rinkuskiai beer sale fell per cent million lit...,1
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,operating profit fell eur mn eur mn including ...,0
4844,negative,Net sales of the Paper segment decreased to EU...,net sale paper segment decreased eur mn second...,0


In [49]:
# Split the cleaned text
X_train, X_test, y_train, y_test = preprocessor.split_data(test_size=0.2)
# Vectorize the text data
(X_train_vec, X_test_vec), vectorizer = preprocessor.vectorize_text()

In [57]:
# Optional: Reduce dimensionality if the vectorized data is very high-dimensional
# This can improve HMM stability
from sklearn.decomposition import TruncatedSVD
n_components = min(40, X_train_vec.shape[1] - 1)  # Ensure we don't exceed the number of features
svd = TruncatedSVD(n_components=n_components, random_state=42)
X_train_reduced = svd.fit_transform(X_train_vec)
X_test_reduced = svd.transform(X_test_vec)
print(f"Reduced data dimensions from {X_train_vec.shape[1]} to {X_train_reduced.shape[1]}")

Reduced data dimensions from 20000 to 40


In [59]:
trainer = ModelTrainer()
hmm_models = trainer.train_hidden_markov_model(X_train_reduced, y_train, n_components=2)
    
# Make predictions
predictor = ModelPredictor()
y_pred = predictor.predict_hidden_markov_model(X_test_reduced, model_trainer=trainer)

# Evaluate the model
print("\nHMM Model Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

HMM models saved at: d:\Tamnd\Tailieudh\Machine learning\ml-course-shibainu\models\trained\hmm_models.pkl

HMM Model Performance:
Accuracy: 0.6113
              precision    recall  f1-score   support

           0       0.23      0.13      0.17       121
           1       0.67      0.93      0.78       576
           2       0.44      0.16      0.24       273

    accuracy                           0.61       970
   macro avg       0.44      0.41      0.39       970
weighted avg       0.55      0.61      0.55       970

